In [1]:
#import libraries
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
import IPython.display as ipd
from IPython.core.display import display
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Softmax,BatchNormalization,GlobalAveragePooling2D,Average
from tensorflow.keras.utils import to_categorical
from keras.models import Model, Input

In [2]:
def create_keras_model_1(model_input, num_classes):
    x = Conv2D(64, kernel_size=3, activation="relu")(model_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(num_classes, activation="softmax")(x)
    model = Model(model_input, x, name='model_1')
    
    return model

In [3]:
def create_keras_model_2(model_input, num_classes):

    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)

    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D(pool_size=(3, 3))(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(model_input, x, name='model_2')

    return model

In [4]:
model_input = Input(shape=(128,216,1))

In [5]:
model_1 = create_keras_model_1(model_input,50)
model_1.load_weights('../input/mel-spectrogram-training-architectures/mel_model_1.h5')

In [6]:
model_2 = create_keras_model_2(model_input,50)
model_2.load_weights('../input/mel-spectrogram-training-architectures/mel_model_2.h5')

In [7]:
models = [model_1,model_2]

In [8]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 216, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 126, 214, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 107, 64)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 105, 128)      73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 52, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 50, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 25, 256)       0   

In [9]:
model_2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 216, 1)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 126, 214, 64)      640       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 214, 64)      256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 42, 71, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 40, 69, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 69, 128)       512       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 38, 67, 128)       1475

In [10]:
def ensemble(models,model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [11]:
ensemble_model = ensemble(models,model_input)

In [12]:
ensemble_model.summary()

Model: "ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 216, 1) 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 126, 214, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 126, 214, 64) 256         conv2d_4[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 42, 71, 64)   0           batch_normalization[0][0]        
___________________________________________________________________________________________

In [13]:
mel_test_data_cat = np.load('../input/test-data-esc-50/mel_test_data_cat.npy',allow_pickle=True)

In [14]:
x_test_mel = np.zeros((1,128,216))
for i in tqdm(range(200)):
    x_test_mel = np.append(x_test_mel,[mel_test_data_cat[i][0][:,:,-1]],axis=0)

100%|██████████| 200/200 [00:01<00:00, 159.69it/s]


In [15]:
x_test_mel = x_test_mel[1:,:,:]

In [16]:
x_test_mel = x_test_mel[:,:,:,np.newaxis]

In [17]:
y_test_mel = []
for i in range(200):
    y_test_mel.append(mel_test_data_cat[i][1])

In [18]:
y_test_mel = np.array(y_test_mel)

In [19]:
len(y_test_mel)

200

In [20]:
arr = ensemble_model.predict(x_test_mel)

In [21]:
y_pred = []
for i in range(200):
    y_pred.append(np.where(arr[i] == np.amax(arr[i]))[0])

In [22]:
np.unique(y_pred)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [23]:
y_pred = np.array(y_pred).flatten()

In [24]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,y_test_mel))

0.92


In [25]:
ensemble_model.save('ensemble_model.h5')